In [ ]:
 
import geopandas as gpd
"""
download rainfall 
"""
ADMIN_PATH = 'data/gis_data/phl_admin3_simpl2.geojson'
list_df=[]  #to store final rainfall dataframes 
#ADMIN_PATH = 'data/gis_data/phl_admin3_simpl2.geojson'
admin = gpd.read_file(ADMIN_PATH)


RAINFALL_TIME_STEP=['06', '24']

url_base = 'https://nomads.ncep.noaa.gov/pub/data/nccf/com/naefs/prod/gefs.'

#url1 = f"{url_base}{data_point}/"  # Use the timestamp of the input folder for the query
#url2 = f"{url_base}{Alternative_data_point}/"  # Yesterday's date
import datetime as dt
import rasterio
import xarray as xr
start_time = dt.datetime.now()
from rasterstats import zonal_stats
no_data_value=29999

import os
import urllib.request
import urllib.error
import requests
import logging
from pathlib import Path
import shutil

def url_is_alive(url):
    """
    Checks that a given URL is reachable.
    :param url: A URL
    :rtype: bool
    """
    request = urllib.request.Request(url)
    request.get_method = lambda: 'HEAD'

    try:
        urllib.request.urlopen(request)
        return True
    except urllib.error.HTTPError:
        return False
    
def get_grib_files(url, path, rainfall_path, use_cache=True):
    base_urls = []
    for items in url: #listFD(url):
        if url_is_alive(items+'prcp_bc_gb2/'):
            base_urls.append(items)
    base_url = base_urls[-1]
    base_url_hour = base_url+'prcp_bc_gb2/geprcp.t%sz.pgrb2a.0p50.bc_' % base_url.split('/')[-2]
    time_step_list1 = [ '24', '30', '36', '42', '48', '54', '60', '66', '72']
    time_step_list = ['06', '12', '18', '24', '30', '36', '42', '48', '54', '60', '66', '72']
    rainfall_24 = [base_url_hour+'24hf0%s' % t for t in time_step_list1]
    rainfall_06 = [base_url_hour+'06hf0%s' % t for t in time_step_list]
    for rain_file in rainfall_06 + rainfall_24:
        output_file = os.path.join(os.path.relpath(rainfall_path, path), rain_file.split('/')[-1]+'.grib2')
        if use_cache and os.path.isfile(output_file):
      
            continue
        try:
            with urllib.request.urlopen(rain_file) as response, open(output_file, 'wb') as out_file:

                shutil.copyfileobj(response, out_file)
        except urllib.error.HTTPError:

            continue

 
data_point = start_time.strftime("%Y%m%d") 
MAIN_DIRECTORY ='./'
rainfall_path =MAIN_DIRECTORY+ 'forecast/rainfall/' 
Alternative_data_point= data_point

try:

    url1=[f'{url_base}{data_point}/{hh}/' for hh in ['00','06','12','18']]
    get_grib_files(url1, MAIN_DIRECTORY, rainfall_path)
except IndexError:
    # If list index out of range then it means that there are no files available,
    # use tomorrow's date instead

    url2=[f'{url_base}{Alternative_data_point}/{hh}/' for hh in ['00','06','12','18']]
    get_grib_files(url2, MAIN_DIRECTORY, rainfall_path)

for hour in RAINFALL_TIME_STEP:
    pattern = f'.pgrb2a.0p50.bc_{hour}h'
    output_filename = f'rainfall_{hour}.nc'
    filename_list = Path(rainfall_path).glob(f'*{pattern}*')
    with xr.open_mfdataset(filename_list, engine='cfgrib',
                            combine="nested", concat_dim=["time"],
                            backend_kwargs={"indexpath": "",
                                            'filter_by_keys': {'totalNumber': 30}}
                            ) as ds:
        filepath = os.path.join(rainfall_path, output_filename)
    
        ds = ds.median(dim='number') #store only the median of the ensemble members
        ds.to_netcdf(filepath)
    #zonal stats to calculate rainfall per manucipality 
    #list_df.append(zonal_stat_rain(filepath,admin))  
    rain_6h=rasterio.open(filepath) 
    band_indexes = rain_6h.indexes
    transform = rain_6h.transform
    all_band_summaries = []
    for b in band_indexes:
        array = rain_6h.read(b)
        band_summary = zonal_stats(
            admin,
            array,
            prefix=f"band{b}_",
            stats="mean",
            nodata=no_data_value,
            all_touched=True,
            affine=transform,
        )
        all_band_summaries.append(band_summary)
    # Flip dimensions
    shape_summaries = list(zip(*all_band_summaries))
    # each list entry now reflects a municipalities, and consists of a dictionary with the rainfall in mm / 6h for each time frame
    final = [{k: v for d in s for k, v in d.items()} for s in shape_summaries]
    # Obtain list with maximum 6h rainfall
    maximum_6h = [max(x.values()) for x in final]
    list_df.append(pd.DataFrame(maximum_6h))
df_rain = pd.concat(list_df,axis=1, ignore_index=True) 
df_rain.columns = ["max_"+time_itr+"h_rain" for time_itr in RAINFALL_TIME_STEP]
df_rain['Mun_Code']=list(admin['adm3_pcode'].values)

df_rain.to_csv(os.path.join(rainfall_path, "rain_data.csv"), index=False)
    
 
